In [ ]:
#@title 1. 掛載 Google Drive { display-mode: "form" }from google.colab import driveimport os#@markdown ### **設定 Google Drive 專案目錄**DRIVE_PROJECT_ROOT = "MyQuantProject" #@param {type:"string"}try:    print("正在嘗試掛載 Google Drive...")    drive.mount('/content/drive', force_remount=True)    gdrive_path = os.path.join('/content/drive/MyDrive', DRIVE_PROJECT_ROOT)    os.environ['GDRIVE_PROJECT_PATH'] = gdrive_path    os.environ['PROJECT_ROOT_PATH'] = gdrive_path    print(f"✅ Google Drive 掛載成功！專案雲端路徑設定為：{gdrive_path}")    for folder in ['data', 'models', 'results']:        folder_path = os.path.join(gdrive_path, folder)        if not os.path.exists(folder_path):            os.makedirs(folder_path)            print(f"📁 已在雲端建立資料夾：{folder_path}")except Exception as e:    os.environ['GDRIVE_PROJECT_PATH'] = ""    print(f"⚠️ 掛載 Google Drive 失敗或未授權。錯誤：{e}")

In [ ]:
#@title 2. 同步 GitHub 專案程式碼 { display-mode: "form" }import os#@markdown ### **1. 設定 GitHub 儲存庫**GITHUB_REPO_URL = "https://github.com/hsp1234-web/qlib.git" #@param {type:"string"}#@markdown ### **2. 輸入分支名稱**GIT_BRANCH = "main" #@param {type:"string"}os.environ['GITHUB_REPO_URL'] = GITHUB_REPO_URLos.environ['GIT_BRANCH'] = GIT_BRANCHrepo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')local_repo_path = os.path.join('/content', repo_name)if os.path.exists(local_repo_path):    print(f"專案資料夾已存在，正在切換至 '{GIT_BRANCH}' 分支並更新...")    %cd {local_repo_path}    !git fetch -q origin    !git checkout -q {GIT_BRANCH}    !git pull -q origin {GIT_BRANCH}    %cd /contentelse:    print(f"專案資料夾不存在，正在從 '{GIT_BRANCH}' 分支下載整個專案...")    !git clone -q --branch {GIT_BRANCH} {GITHUB_REPO_URL}print(f"\n✅ 程式碼同步完成！")

In [ ]:
#@title 3. 智慧安裝 Python 套件 (使用 uv) { display-mode: "form" }import osimport sysimport pkg_resourcesimport importlibprint("📦 正在檢查並安裝 uv...")!pip install -q --no-warn-script-location uvprint("✅ uv 已準備就緒。")GITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')if not GITHUB_REPO_URL:    raise ValueError("錯誤：找不到 GITHUB_REPO_URL 環境變數。請先執行儲存格 2。")repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')requirements_path = os.path.join('/content', repo_name, 'requirements.txt')if not os.path.exists(requirements_path):    print(f"⚠️ 警告：在 '{requirements_path}' 中找不到 requirements.txt 檔案。")else:    print("\n🔍 正在分析所需套件...")    importlib.reload(pkg_resources)    installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}    to_install = []    with open(requirements_path, 'r') as f:        for line in f:            line = line.strip()            if not line or line.startswith('#'): continue            try:                req = pkg_resources.Requirement.parse(line)                if req.key not in installed_packages or not req.specifier.contains(installed_packages.get(req.key, '0.0.0')):                    to_install.append(line)            except pkg_resources.RequirementParseError:                to_install.append(line)    if not to_install:        print("\n✅ 所有 Python 套件皆已滿足。")    else:        print(f"\n🚀 偵測到 {len(to_install)} 個需要安裝/更新的套件，開始安裝...")        packages_str = " ".join(f'\"{pkg}\"' for pkg in to_install)        !uv pip install --quiet {packages_str}        print("\n✅ 環境安裝完成！")

In [ ]:
#@title 4. 執行回測 (指令列模式) { display-mode: "form" }import os#@markdown ### **回測參數設定**TICKER = "^TWII" #@param {type:"string"}START_DATE = "2020-01-01" #@param {type:"date"}END_DATE = "2023-01-01" #@param {type:"date"}STRATEGY = "defaultlong" #@param ["defaultlong", "defaultshort", "defaultall"]GITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')if not GITHUB_REPO_URL:    raise ValueError("錯誤：找不到 GITHUB_REPO_URL。")repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')script_path = f"/content/{repo_name}/src/backtest_runner.py"if not os.path.exists(script_path):    raise FileNotFoundError(f"錯誤：找不到回測腳本 '{script_path}'。")command = f"python {script_path} --ticker \"{TICKER}\" --start_date \"{START_DATE}\" --end_date \"{END_DATE}\" --strategy {STRATEGY}"print(f"執行命令: {command}")!{command}print("\n✅ 回測執行完畢！")

In [ ]:
#@title 4.1 (新功能) 啟動互動式回測設定儀表板 { display-mode: "form" }# ==============================================================================# 目的：# 啟動一個網頁圖形化介面 (GUI)，讓您可以更直觀地設定參數並執行回測。## 執行動作：# 1. 啟動伺服器：在背景執行 Gradio 網頁應用程式。# 2. 產生公開網址：Gradio 會自動產生一個可分享的公開網址 (URL)。## 如何使用：# 1. 執行此儲存格。# 2. 在輸出區找到類似 `Running on public URL: https://....gradio.live` 的網址。# 3. **點擊該網址**，會在新的瀏覽器分頁中開啟儀表板介面。# 4. 在新開啟的網頁中設定參數並執行回測。# ==============================================================================import osGITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')if not GITHUB_REPO_URL:    raise ValueError("錯誤：找不到 GITHUB_REPO_URL。請先執行儲存格 2。")repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')app_path = f"/content/{repo_name}/lo2cin4bt/app.py"if not os.path.exists(app_path):    raise FileNotFoundError(f"錯誤：找不到應用程式腳本 '{app_path}'。")# 使用 -m 將 lo2cin4bt 作為模組執行，以確保相對匯入正確# 我們需要先進入專案資料夾，才能正確地以模組方式執行%cd /content/{repo_name}!python -m lo2cin4bt.app

In [ ]:
#@title 5. (可選) 同步成果至 Google Drive { display-mode: "form" }import osfrom google.colab import drivefrom distutils.dir_util import copy_treegdrive_path = os.environ.get('GDRIVE_PROJECT_PATH')project_root = os.environ.get('PROJECT_ROOT_PATH')if not project_root or not os.path.exists(project_root):    print("❌ 錯誤：找不到任何本地專案成果。")elif gdrive_path and project_root == gdrive_path:    print("✅ 您的專案成果已經儲存在 Google Drive 中，無需同步。")else:    print("🔄 開始將本地成果同步至 Google Drive...")    if not gdrive_path:        try:            print("正在嘗試掛載 Google Drive...")            drive.mount('/content/drive', force_remount=True)            project_name = "MyQuantProject"            gdrive_path = os.path.join('/content/drive/MyDrive', project_name)            os.environ['GDRIVE_PROJECT_PATH'] = gdrive_path            print(f"✅ Google Drive 掛載成功！")        except Exception as e:            print(f"❌ 掛載 Google Drive 失敗。錯誤：{e}")            gdrive_path = None    if gdrive_path:        print(f"正在將 '{project_root}' 複製到 '{gdrive_path}'...")        os.makedirs(gdrive_path, exist_ok=True)        copy_tree(project_root, gdrive_path)        print("✅ 同步完成！")